<a href="https://colab.research.google.com/github/LilySu/BetterBusinessByReview/blob/master/Word_Phrase_Endpoint_Refactoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scattertext
!pip install -U spacy
!python -m spacy download en_core_web_sm

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.3)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [1]:
import json
import warnings
import pandas as pd
import spacy
import scattertext as st
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)
nlp = spacy.load("en_core_web_sm")#if you run into problems here, 'Restart Runtime' and run all, it might fix things.
base_url = "https://www.yelp.com/biz/" 
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'Rc1lxc5lSKJYd162JHNMfQ'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data


def customtokensize(text):
    return re.findall("[\w']+", str(text))

df['tokenized_text'] = df[1].apply(customtokensize)
df['parts_of_speech_reference'] = df['tokenized_text'].str.join(' ')
def find_word_segment_combined(x):
  long_phrase_list = []
  doc = nlp(str(x))
  try:
    for token in range(len(doc)):
      sub_list = []
      if (token < (int(len(doc))-5)) and ((doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN') or (doc[token].text == 'food') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN')):
        sub_list.append(doc[token])
        sub_list.append(doc[token+1])
        sub_list.append(doc[token+2])
        sub_list.append(doc[token+3])
        sub_list.append(doc[token+4])
        sub_list.append(doc[token+5])
        sub_list.append(doc[token+6])
      if len(sub_list) != 0:
        long_phrase_list.append(sub_list)
  except IndexError as e:
    try:
      for token in range(len(doc)):
        sub_list = []
        if (token < (int(len(doc))-5)) and ((doc[token].pos_ == 'PRON') or ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An')):
          sub_list.append(doc[token])
          sub_list.append(doc[token+1])
          sub_list.append(doc[token+2])
          sub_list.append(doc[token+3])
          sub_list.append(doc[token+4])
          sub_list.append(doc[token+5])
        if len(sub_list) != 0:
          long_phrase_list.append(sub_list)
    except IndexError as e:
      try:
        word_list1 = []
        if (doc[token].pos_ == 'PRON') or ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An'):
          word_list1.append(doc[token])
          word_list1.append(doc[token+1])
          word_list1.append(doc[token+2])
          word_list1.append(doc[token+3])
          word_list1.append(doc[token+4])
          word_list1.append(doc[token+5])
          word_list1.append(doc[token+6])
          word_list1.append(doc[token+7])
          word_list1.append(doc[token+8])
          word_list1.append(doc[token+9])
          word_list1.append(doc[token+10])
          word_list1.append(doc[token+11])
        if len(word_list1) != 0:
          long_phrase_list.append(word_list1)
      except IndexError or TypeError as e:
        for token in range(len(doc)):
          try:
            word_list2 = []
            if (doc[token].pos_ == 'PRON') or ((doc[token].pos_ == 'the') and (doc[token].pos_ == 'PRON')) or (doc[token].text == 'The') or (doc[token].text == 'a') or (doc[token].text == 'A') or (doc[token].text == 'an') or (doc[token].text == 'An'):
              word_list2.append(doc[token])
              word_list2.append(doc[token+1])
              word_list2.append(doc[token+2])
              word_list2.append(doc[token+3])
              word_list2.append(doc[token+4])
              word_list2.append(doc[token+5])
              word_list2.append(doc[token+6])
              word_list2.append(doc[token+7])
            if (len(word_list2) != 0) and (False for i in word_list2 for i in word_list1) and (False for i in word_list2 for i in long_phrase_list):
              long_phrase_list.append(word_list2) 
          except IndexError or TypeError as e:
            for token in range(len(doc)):
              try:
                word_list3 = []
                if ((doc[token-1].pos_ == 'VERB') and (doc[token].text == 'the')) or ((doc[token-1].pos_ == 'ADV') and (doc[token].text == 'the')):
                  word_list3.append(doc[token-1])
                  word_list3.append(doc[token])
                  word_list3.append(doc[token+1])
                  word_list3.append(doc[token+2])
                if (len(word_list3) != 0 ) and (False for i in word_list3 for i in word_list2) and (False for i in word_list3 for i in long_phrase_list):
                  long_phrase_list.append(word_list3)
              except IndexError or TypeError as e:
                for token in range(len(doc)):
                  try:
                    word_list4 = []
                    if (doc[token].text == 'the') or (doc[token].text == 'a') or (doc[token].text == 'an'):
                      word_list4.append(doc[token])
                      word_list4.append(doc[token+1])
                      word_list4.append(doc[token+2])
                      word_list4.append(doc[token+3])
                    if (len(word_list4) != 0 ) and (False for i in word_list4 for i in word_list3) and (False for i in word_list4 for i in long_phrase_list):
                      long_phrase_list.append(word_list4)
                  except IndexError or TypeError as e:
                    try:
                      word_list5 = []
                      if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                        word_list5.append(doc[token-1])
                        word_list5.append(doc[token])
                        word_list5.append(doc[token+1])
                        word_list5.append(doc[token+2])
                        word_list5.append(doc[token+3])
                      if (len(word_list5) != 0) and (False for i in word_list5 for i in word_list4) and (False for i in word_list5 for i in long_phrase_list):
                        long_phrase_list.append(word_list5) 
                    except IndexError as e:
                      try:
                        word_list6 = []
                        if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                          word_list6.append(doc[token-1])
                          word_list6.append(doc[token])
                          word_list6.append(doc[token+1])
                          word_list6.append(doc[token+2])
                        if (len(word_list6) != 0) and (False for i in word_list6 for i in word_list5) and (False for i in word_list6 for i in long_phrase_list):
                          long_phrase_list.append(word_list6) 
                      except IndexError as e:
                        try:
                          word_list7 = []
                          if (doc[token-1].text == 'on') and (doc[token].text == 'a') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                            word_list7.append(doc[token-1])
                            word_list7.append(doc[token])
                            word_list7.append(doc[token+1])
                          if (len(word_list7) != 0) and (False for i in word_list7 for i in word_list6) and (False for i in word_list7 for i in long_phrase_list):
                            long_phrase_list.append(word_list7) 
                        except IndexError as e:
                          try:
                            word_list8 = []
                            if (doc[token].text == 'on') or (doc[token].text == 'for') or (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'VERB' and doc[token+1].pos_ == 'NOUN') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
                              word_list8.append(doc[token])
                              word_list8.append(doc[token+1])
                            if (len(word_list8) != 0) and (False for i in word_list8 for i in word_list7) and (False for i in word_list8 for i in long_phrase_list):
                              long_phrase_list.append(word_list8) 
                          except IndexError as e:
                            try:
                              word_list9 = []
                              #captures words at the end of reviews
                              if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token-1].pos_ == 'NOUN' and doc[token].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token-1].pos_ == 'ADJ'and doc[token].pos_ =='NOUN') or (doc[token-1].pos_ == 'VERB'and doc[token].pos_ =='NOUN')or (doc[token-1].pos_ == 'NOUN'and doc[token].pos_ =='NOUN'):
                                word_list9.append(doc[token-1])
                                word_list9.append(doc[token])
                              if (len(word_list9) != 0) and (False for i in word_list9 for i in word_list8) and (False for i in word_list9 for i in long_phrase_list):
                                long_phrase_list.append(word_list9) 
                            except IndexError as e:
                              try:
                                word_list10 = []
                                #captures words from the start of reviews
                                if (doc[token].text == 'food is') or (doc[token].lemma_ == 'order') or (doc[token].text == 'but') or (doc[token].text == 'not') or (doc[token].pos_ == 'NOUN' and doc[token+1].pos_ == 'VERB') or (doc[token].lemma_ == 'service') or (doc[token].lemma_ == 'wait') or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN')or (doc[token].pos_ == 'NOUN'and doc[token+1].pos_ =='NOUN'):
                                  word_list10.append(doc[token])
                                  word_list10.append(doc[token+1])
                                if (len(word_list10) != 0) and (False for i in word_list10 for i in word_list9) and (False for i in word_list10 for i in long_phrase_list):
                                  long_phrase_list.append(word_list10) 
                              except IndexError as e:
                                try:
                                  word_list11 = []
                                  #captures words from the start of reviews
                                  word_list11.append(doc[token])
                                  word_list11.append(doc[token+1])
                                  if (len(word_list11) != 0) and (False for i in word_list10 for i in word_list10) and (False for i in word_list11 for i in long_phrase_list):
                                    long_phrase_list.append(word_list11) 
                                except IndexError as e:
                                  pass
  return long_phrase_list

df['word_segments'] = df[1].apply(find_word_segment_combined)
df['word_segments_unpacked'] = df['word_segments'].apply(lambda l: [item for sublist in l for item in sublist])
df['word_segments_unpacked'] = df['word_segments_unpacked'].astype(str)
df['word_segments_unpacked'] = df['word_segments_unpacked'].apply(lambda x: ''.join([str(i) for i in x]))
phrase_count = df[['word_segments_unpacked', 2]]
import numpy as np

s= phrase_count.apply(lambda x: pd.Series(x['word_segments_unpacked']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'word_segments_unpacked'

phrase_count = phrase_count.drop('word_segments_unpacked', axis=1).join(s)
phrase_count = pd.DataFrame(df['word_segments_unpacked'].str.split(',').tolist(), index=df[2]).stack()
phrase_count = phrase_count.reset_index()[[0, 2]] # var1 variable is currently labeled 0
phrase_count.columns = ['word_segments_unpacked', 'ratings'] # renaming var1
phrase_count = phrase_count.reset_index(drop=False)
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace('[','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace(']','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace('-','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace('.','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.replace(' ','')
phrase_count['word_segments_unpacked'] = phrase_count['word_segments_unpacked'].str.lower()

stopwords = ['0','1','2','3','4','5','6','7','8','9','/','$',"'d","'ll","'m",'+','maybe','from','first','here','only','put','where','got','sure','definitely','food','yet','our','go','since','really','very','two',"n't",'with','if',"'s",'which','came','all','me','(',')','makes','make','were','immediately','get','been','ahead','also','that','one','have','see','what','to','we','had','.',"'re",'it','or','he','she','we','us','how','went','no','"','of','has','by','bit','thing','place','so','ok','and','they','none','was','you',"'ve",'did','be','and','but','is','as','&','you','has','-',':','and','had','was','him','so','my','did','would','her','him','it','is','by','bit','thing','place','[',']','while','check-in','=','= =','want', 'good','husband', 'want','love','something','your','they','your','cuz','him',"i've",'her','told', 'check', 'i"m', "it's",'they', 'this','its','they','this',"don't",'the',',', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.']
def filter_stopwords(text):
  for i in str(text):
    if i not in stopwords:
      return str(text)

phrase_count = phrase_count[~phrase_count['word_segments_unpacked'].isin(stopwords)]#if item in stopwords list partially matches, delete

#full matches
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='i']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='a']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='an']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='am']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='at']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='are']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='in']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='on']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='for']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='\xa0\xa0']
phrase_count = phrase_count[phrase_count['word_segments_unpacked']!='\xa0']

phrase_count_pivot = pd.pivot_table(phrase_count, index='word_segments_unpacked', columns='ratings', aggfunc='count', fill_value=0)
phrase_count_pivot.columns = [''.join(col).strip() for col in phrase_count_pivot.columns.values]#flatten index levels part 1
phrase_count_pivot = pd.DataFrame(phrase_count_pivot.to_records())#flatten index levels part 2

#scale
phrase_count_pivot['index1.0 star rating'] = phrase_count_pivot['index1.0 star rating']*(-2)
phrase_count_pivot['index2.0 star rating'] = phrase_count_pivot['index2.0 star rating']*(-1)
phrase_count_pivot['index3.0 star rating'] = phrase_count_pivot['index3.0 star rating']*(-0.1)
phrase_count_pivot['index4.0 star rating'] = phrase_count_pivot['index4.0 star rating']*(1)
phrase_count_pivot['index5.0 star rating'] = phrase_count_pivot['index5.0 star rating']*(2)


phrase_count_pivot['score'] = phrase_count_pivot['index1.0 star rating'] + phrase_count_pivot['index2.0 star rating'] + phrase_count_pivot['index3.0 star rating'] + phrase_count_pivot['index4.0 star rating'] + phrase_count_pivot['index5.0 star rating']

phrase_count_pivot['score'] = phrase_count_pivot['score'].div(phrase_count_pivot['score'].max(), axis=0)
phrase_count_pivot['score'] = phrase_count_pivot['score'].round(decimals=4)
phrase_count_pivot = phrase_count_pivot.sort_values(by=('score'), ascending=False)
phrase_count_pivot.head(2)

,word_segments_unpacked,index1.0 star rating,index2.0 star rating,index3.0 star rating,index4.0 star rating,index5.0 star rating,score
401,great,-8,-6,-0.5,30,166,1.0000
798,service,-26,-11,-1.3,30,82,0.4061


In [2]:
phrase_count_pivot['word_segments_unpacked'].iloc[-1]

'lukewarm'

In [17]:
worst_terms_list = []
for i in range(-10, 0):
  new_df = df[df['word_segments_unpacked'].str.contains(phrase_count_pivot['word_segments_unpacked'].iloc[i])]
  worst_terms_list.append(str(new_df['word_segments_unpacked']))
worst_terms_list

["2    [Great, atmosphere, ,, great, food, ,, TERRIBLE, great, food, ,, TERRIBLE, service, all, around, food, ,, TERRIBLE, service, all, around, from, service, all, around, from, the, front, door, front, door, to, the, table, ., \xa0, local, restos, in, HK, ,, but, tough, but, tough, one, to, come, back, to, tough, one, to, come, back, to, let]\n4                                                 [great, service, sitting, at, the, bar, on, service, sitting, at, the, bar, on, a, sitting, at, the, bar, on, a, Sunday, Good, cocktail, selection, -, I, enjoyed, the, great, restaurants, around, ,, I, 'm, not, not, sure, if, I, 'll, return, anytime, but, I, had, a, positive, experience, .]\n2                                                                                                     [Poor, service, ., Indifferent, wait, staff, just, service, ., Indifferent, wait, staff, just, standing, Indifferent, wait, staff, just, standing, around, ., wait, staff, just, standing, around, ., Two, hot,

In [27]:
for i in range(-10, 0):
  for list_of_words in worst_terms_list:
    word_list = list_of_words.split(', ')
    for word in word_list:
      if phrase_count_pivot['word_segments_unpacked'].iloc[i] == word:
        index = word_list.index(word)
        # print(index)
        rejoined_word_list = ','.join(word_list[max(0,index-5):min(index+10, len(word_list))])
        rejoined_word_list=rejoined_word_list.replace(' ', '')
        rejoined_word_list=rejoined_word_list.replace(',', ' ')
        rejoined_word_list=rejoined_word_list.replace('.', '')
  print(rejoined_word_list)
  

Indifferent wait staff just standing around  wait staff just standing around  Two hot
 Indifferent wait staff just standing Indifferent wait staff just standing around  wait staff
I ordered a ordered a seltzer with pomegranate juice and pomegranate juice and I got
I ordered a ordered a seltzer with pomegranate juice and pomegranate juice and I got
I have been to  arrived we were seated in a booth orders   our
and we were still order gave us something we did n't gave us something we
food  The service was distant food  The service was distant   service was
our drinks were delivered with minimal taste and minimal alcohol    minimal alcohol 
asked for a straw that wait for our entrees for quite some but the bartender
after we complained food was lukewarm at best    eating leftovers and too boot
